oracle telling us if decryption of ciphertext using key=nonce+flag is 'ping' \
we can get keystream since we have c and m, so we're supposed to test keystream obv \
s{nonce+flag} \
ok so oracle leaks info on whether last char of ground nonce == first char of flag \
ez \
ok not really, forgot about things like, of course rc4 doesn't truncate the key what are you on rn \
although we do have an oracle for if nonce==FLAG. that's something \
ok found the attack -_- easy \
try implement it from scratch though \

In [5]:
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta
from Crypto.Cipher import AES
from os import urandom
import requests
from Crypto.Util.number import long_to_bytes, bytes_to_long
import json
from tqdm import tqdm
import re
def json_get(url):
    return json.loads(requests.get(url).content.decode())
def xor(a, b):
    return long_to_bytes(bytes_to_long(a) ^ bytes_to_long(b))
def repeat_xor(a, b):
    return bytes(x ^ y for x,y in zip(a, b * (1 + len(a) // len(b))))
def decrypt(ciphertext, nonce):
    return bytes.fromhex(json_get(f"https://aes.cryptohack.org/oh_snap/send_cmd/{ciphertext}/{nonce}/")['error'].split(": ")[1])
def is_utf8(s: str):
    try:
        s.decode('utf-8')
        return True
    except UnicodeError:
        return False
KEY =  b"key{the_aes_key}"
FLAG = b"crypto{aes_flag}"
PLAIN =b"Howdy! I'm Flowe"
# IV = b'\xda\xdd:\x7f2\xa7\xc2\xe3"\xbff\xca\r\x0c?\xcc'

In [42]:
xor(decrypt(FLAG.hex(), PLAIN[:8].hex()), FLAG)

b'Q\x86\xe0\xb7\xefM\xb8T\xa0\xaf\x1f!\x19{\xcd}'

In [68]:
from Crypto.Cipher import ARC4
ce = ARC4.new(b"flag")
print(xor(ce.encrypt(b"\x03\xff\x00"+PLAIN), PLAIN))
ce = ARC4.new(b"flag")
print(xor(ce.encrypt(b"\x03\xff\x00"+FLAG), FLAG))

b'\xec\x0f\xd4\t\xcb\xe6K0\xbaY\xd5Xp\x8f\xe8n\xc9\x90&'
b'\xec\x0f\xd4\t\xcb\xe6K0\xbaY\xd5Xp\x8f\xe8n\xc9\x90&'


In [28]:
# soln
from ctflib import pool
from collections import Counter
cnter = Counter()
def test(c: int):
    from ctflib import rc4_fms_possible_key_bit, decrypt
    known = b"crypto{w1R3d_equ1v4l3nt_pr1v4cy?!}"
    key = bytes([3+len(known)])+b"\xff"+bytes([c])+known
    p = rc4_fms_possible_key_bit(key, decrypt(b"\x00".hex(), key[:3].hex()))# \x00 chosen so output is the cipherstream
    return chr(p)
[cnter.update(c) for c in pool(test, 256, 16)]
cnter

  0%|          | 0/256 [00:00<?, ?it/s]

Counter({'}': 8,
         'q': 5,
         's': 4,
         '\x8c': 4,
         '\x98': 4,
         'Ç': 4,
         'ù': 4,
         '\x13': 4,
         '\x9e': 3,
         'F': 3,
         '\x90': 3,
         '\x9b': 3,
         '\x84': 3,
         '.': 3,
         '\x10': 3,
         'ì': 3,
         '£': 2,
         'f': 2,
         '\x12': 2,
         '°': 2,
         '¹': 2,
         'w': 2,
         '¾': 2,
         '\x01': 2,
         'à': 2,
         '?': 2,
         '\x8e': 2,
         '\x7f': 2,
         "'": 2,
         '\x04': 2,
         '\x87': 2,
         '|': 2,
         '¸': 2,
         'I': 2,
         'è': 2,
         '©': 2,
         'Þ': 2,
         'Q': 2,
         '#': 2,
         'ÿ': 2,
         '\xad': 2,
         'z': 2,
         'h': 2,
         '\x89': 2,
         '9': 2,
         'á': 2,
         'î': 2,
         'Û': 2,
         '\x14': 2,
         'Ø': 2,
         ',': 2,
         '²': 2,
         ' ': 2,
         't': 2,
         '\x86': 2,
         'é

In [95]:
from Crypto.Cipher import ARC4
from collections import Counter
def _possible_key_bit(key, c):
    s = [i for i in range(256)]
    j = 0
    for i in range(len(key)):
        j = (j + s[i] + key[i]) % 256
        tmp = s[i]
        s[i] = s[j]
        s[j] = tmp
    return (c[0] - j - s[len(key)]) % 256

cnter = Counter()
for c in tqdm(range(0,256)):
    key = b"\x03\xff"+bytes([c])
    p = _possible_key_bit(key, decrypt(b"\x00".hex(), key.hex()))
    cnter.update(chr(p))
cnter

100%|██████████| 256/256 [02:11<00:00,  1.95it/s]


Counter({'c': 14,
         '7': 5,
         'ð': 4,
         'Y': 4,
         '?': 4,
         '\x17': 4,
         '¬': 4,
         'b': 3,
         '(': 3,
         '¦': 3,
         '\x90': 3,
         '\x8f': 3,
         'Í': 3,
         'Â': 3,
         'q': 3,
         'B': 3,
         '\x04': 3,
         '´': 3,
         'f': 3,
         '=': 2,
         'j': 2,
         '3': 2,
         '\x06': 2,
         '\x95': 2,
         '¶': 2,
         'ç': 2,
         's': 2,
         '\x83': 2,
         '^': 2,
         'ø': 2,
         '8': 2,
         '[': 2,
         '¹': 2,
         'O': 2,
         '@': 2,
         '\x11': 2,
         '½': 2,
         '\x19': 2,
         'î': 2,
         ']': 2,
         'o': 2,
         '\x81': 2,
         '>': 2,
         'Ã': 2,
         'Æ': 2,
         'D': 2,
         'Ñ': 2,
         '%': 2,
         '<': 2,
         '\x9f': 2,
         'µ': 2,
         '\x1a': 2,
         '\x0f': 2,
         '¼': 2,
         'Ò': 2,
         'Q': 2,
        